In [1]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Carpeta root del dataset
image_path = 'BD_perros'

transform = transforms.Compose([transforms.ToTensor(),
                                #transforms.Resize((28,28)),
                                ])

dataset = torchvision.datasets.ImageFolder(image_path, transform=transform)

In [5]:
len(dataset)

1030

In [7]:
batch_size = 64

num_datos = len(dataset)
num_train = int(0.7 * num_datos)
num_test = num_datos-num_train

train_dataset = Subset(dataset, torch.arange(num_train))
test_dataset  = Subset(dataset, torch.arange(num_train, len(dataset)))

train_dl = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)
valid_dl = DataLoader(test_dataset , batch_size= batch_size, shuffle= False)

In [8]:
model = nn.Sequential()
# Conv -> ReLU -> MaxPooling
model.add_module('conv1', nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,padding=2))
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
# Conv -> ReLU -> MaxPooling
model.add_module('conv2', nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,padding=2))
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))
# Flatten
model.add_module('flatten', nn.Flatten())
# Full Connected -> ReLU -> Dropout
model.add_module('fc1', nn.Linear(3136,1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p= .5))
# Full Connected
model.add_module('fc2', nn.Linear(1024, 10))
#model.add_module('soft', nn.Softmax(dim=10))

In [9]:
# Selección de la unidad de procesamiento
processing_unit = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(processing_unit)
model.to(device)
print(f"You're using: {device} as device.")

You're using: cuda as device.


In [13]:
import torchsummary
torchsummary.summary(model,input_data= torch.randint(0, 255, (batch_size, 3, 28, 28))/ 255,
col_names=["output_size", "num_params"], verbose= 0,device= device)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 32, 28, 28]          2,432
├─ReLU: 1-2                              [-1, 32, 28, 28]          --
├─MaxPool2d: 1-3                         [-1, 32, 14, 14]          --
├─Conv2d: 1-4                            [-1, 64, 14, 14]          51,264
├─ReLU: 1-5                              [-1, 64, 14, 14]          --
├─MaxPool2d: 1-6                         [-1, 64, 7, 7]            --
├─Flatten: 1-7                           [-1, 3136]                --
├─Linear: 1-8                            [-1, 1024]                3,212,288
├─ReLU: 1-9                              [-1, 1024]                --
├─Dropout: 1-10                          [-1, 1024]                --
├─Linear: 1-11                           [-1, 10]                  10,250
Total params: 3,276,234
Trainable params: 3,276,234
Non-trainable params: 0
Total mult-adds (M): 15.14
Input size (MB): 0.57
Forward/back

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= .001)

In [15]:
def train(model, num_epochs, train_dl, valid_dl, device):
    loss_hist_train = torch.zeros(num_epochs).to(device)
    accuracy_hist_train = torch.zeros(num_epochs).to(device)
 
    loss_hist_valid = torch.zeros(num_epochs).to(device)
    accuracy_hist_valid = torch.zeros(num_epochs).to(device)
 
    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item() * y_batch.size(0)
            is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
            accuracy_hist_train[epoch] += is_correct.sum()

        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                loss_hist_valid[epoch] += loss.item() * y_batch.size(0)
                is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
                accuracy_hist_valid[epoch] += is_correct.sum()
            loss_hist_valid[epoch] /= len(valid_dl.dataset)
            accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
        print(f'Epoch {epoch+1} accuracy: {accuracy_hist_train[epoch]:.4f} 'f'val_accuracy: {accuracy_hist_valid[epoch]:.4f}')
    return loss_hist_train.cpu(), loss_hist_valid.cpu(), accuracy_hist_train.cpu(), accuracy_hist_valid.cpu()

In [16]:
torch.manual_seed(1)
num_epochs = 4
hist = train(model, num_epochs, train_dl, valid_dl, device)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x200704 and 3136x1024)